In [18]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))

import config

In [25]:
# # Configura el entorno de trabajo
%reload_ext autoreload
%autoreload 2

In [20]:
import hopsworks

# Conecta a Hopsworks
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# Conecta a la API de Feature Store
feature_store = project.get_feature_store()

# Conecta al grupo de características
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2025-05-08 18:12:05,782 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 18:12:05,795 INFO: Initializing external client
2025-05-08 18:12:05,796 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 18:12:06,955 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224869


In [ ]:
# Crea una vista de características (si no existe ya)
# Esta vista de características usa solo un grupo de características, por lo que la consulta es trivial
try:
    # Crea la vista de características si no existe
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')

Feature view already existed. Skip creation.


In [ ]:
# obtenemos la vista de características
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

In [22]:
# obtenemos la vista de características necesaria para el entrenamiento
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.09s) 


In [23]:
ts_data.head()

,pickup_hour,pickup_location_id,rides
0,2024-10-16 16:00:00+00:00,118,0
1,2024-06-30 06:00:00+00:00,226,9
2,2024-07-25 09:00:00+00:00,132,163
3,2024-06-15 16:00:00+00:00,138,163
4,2024-09-19 03:00:00+00:00,246,35


In [26]:
from data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 263/263 [00:32<00:00,  8.02it/s]


features_and_target.shape=(116511, 675)


In [33]:
import pandas as pd

# Convertimos la columna de fecha y hora a un objeto datetime

features_and_target['pickup_hour'] = pd.to_datetime(features_and_target['pickup_hour'])

In [ ]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from data_split import train_test_split

# training data -> from January 2025 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)


# eliminamos la columna de fecha y hora
X_train = X_train.drop(columns=['pickup_hour'])
X_test = X_test.drop(columns=['pickup_hour'])

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2025-04-10 00:00:00+0000', tz='UTC')
X_train.shape=(109145, 674)
y_train.shape=(109145,)
X_test.shape=(7366, 674)
y_test.shape=(7366,)


In [44]:
y_train.tail()

109140    3.0
109141    5.0
109142    3.0
109143    4.0
109144    4.0
Name: target_rides_next_hour, dtype: float32

In [ ]:
# entrenamos un modelo random forest
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint, uniform


# Define el modelo base
rf = RandomForestRegressor(random_state=42)

# 5. Define la distribución de hiperparámetros
param_dist = {
    "n_estimators": randint(50, 300),
    "max_depth": randint(3, 20),
    "min_samples_split": randint(2, 10),
    "min_samples_leaf": randint(1, 4),
    "max_features": uniform(0.5, 0.5),  # uniform(a, b) donde a=0.5, b=0.5 → [0.5,1.0]
}

# 6. Configura RandomizedSearchCV
n_iter_search = 5
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=n_iter_search,
    scoring="neg_mean_squared_error",
    cv=3,
    random_state=42,
    n_jobs=-1,        # usa todos los núcleos disponibles
    verbose=2
)

# Ejecuta la búsqueda
random_search.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf_model.predict(X_test)

# Evaluación
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = mean_squared_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)

print(f"Random Forest - MAE: {mae_rf:.2f} | RMSE: {rmse_rf:.2f} | R2: {r2:.2f}")

In [ ]:
import joblib
from paths import MODELS_DIR

joblib.dump(rf_model, MODELS_DIR / 'model.pkl')

In [48]:
import joblib
from paths import MODELS_DIR
# cargamos el modelo
rf_model = joblib.load(MODELS_DIR / 'random_forest.pkl')

In [ ]:
X_test.columns

X_test = X_test.drop(columns=['pickup_hour','pickup_location_id'])

In [59]:
y_test.describe()

count    7366.000000
mean       16.654629
std        52.034935
min         0.000000
25%         0.000000
50%         0.000000
75%         3.000000
max       713.000000
Name: target_rides_next_hour, dtype: float64

In [52]:
predictions = rf_model.predict(X_test)

mean_absolute_error(y_test, predictions)

34.53512218300298

In [58]:
# creamos el model registry
# el modelo se registrará en el registro de modelos
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from sklearn.metrics import mean_absolute_error

# creamos el esquema de entrada y salida
# el esquema de entrada es el esquema de las características
# el esquema de salida es el esquema de la variable objetivo
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
test_mae = mean_absolute_error(y_test, predictions)

# obtenerlo el model registry - Hopsworks
model_registry = project.get_model_registry()


# registramos el modelo en el registro de modelos
model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="Random Forest regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR / 'random_forest.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/models/random_forest.pkl: 0.000%| …

Uploading /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/notebooks/input_example.json: 0.00…

Uploading /Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/notebooks/model_schema.json: 0.000…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1224869/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)